In [1]:
import sys
!{sys.executable} -m pip install sentencepiece
!{sys.executable} -m pip install torchtext==0.2.3
!conda list --prefix {sys.prefix}

# packages in environment at C:\Users\Getek\Anaconda3\envs\fastai:
#
# Name                    Version                   Build  Channel
asn1crypto                0.24.0                   py36_0  
atomicwrites              1.3.0                    py36_1  
attrs                     19.1.0                   py36_1  
backcall                  0.1.0                    py36_0  
backports                 1.0                      py36_1  
backports.os              0.1.1                    py36_0  
bcolz                     1.2.1            py36h830ac7b_0  
beautifulsoup4            4.6.3                    pypi_0    pypi
blas                      1.0                         mkl  
bleach                    3.1.0                    py36_0  
blosc                     1.16.3               h7bd577a_0  
bokeh                     1.2.0                    py36_0  
bzip2                     1.0.6                hfa6e2cd_5  
ca-certificates           2019.1.23                     0  
certifi           

In [3]:
from fastai.text import *

In [6]:
h5_filepath = "./work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
model_filepath = "./work/up_low50k/tmp/sp-50k.model"

In [7]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(model_filepath)


spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")

True

In [8]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [9]:
print(torch.cuda.device_count())

torch.cuda.device(0)
torch.cuda.current_device()

0


AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [10]:
UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],h5_filepath)
lm.reset()
lm.eval()

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [11]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self):
        return len(self.x)

In [12]:
def next_tokens(ids_, model, deterministic, omit_ids=[]):
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(test_ds,
                    bs,
                    transpose=True,
                    transpose_y=True,
                    num_workers=1,
                    pad_idx=PAD_ID,
                    sampler=test_samp,
                    pre_pad=False)

    tensor1 = None
    with no_grad_context():
        for (x, y) in dl:
            tensor1 = model(x)
    p = tensor1[0]

    arg = p[-1]
    r = int(
        torch.argmax(arg) if deterministic else torch.
        multinomial(p[-1].exp(), 1)) # probability is in logharitm

    while r in omit_ids + [ids_[-1]]:  #, BOS_ID,EOS_ID, UNK_ID]:
        arg[r] = -1
        r = int(torch.argmax(arg))

    predicted_ids = [r]
    return predicted_ids

In [13]:
def next_words_best(ss, lm, n_words, deterministic=True, omit_ids=[]):
    ss_ids = spm_processor.encode_as_ids(ss)
    wip = ss
    wip_ids = ss_ids
    for i in range(n_words):
        next_ = next_tokens(wip_ids, lm, deterministic, omit_ids=omit_ids)
        wip_ids = wip_ids + next_
        wip = spm_processor.decode_ids(wip_ids)
        wip_ids = spm_processor.encode_as_ids(wip)

    return wip

In [14]:
sentences = [
    "Warszawa to największe", "Te zabawki należą do",
    "Policjant przygląda się", "Na środku skrzyżowania widać",
    "Właściciel samochodu widział złodzieja z",
    "Prezydent z premierem rozmawiali wczoraj o", "Witaj drogi",
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie",
    "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"
]

In [15]:
def prediction(sentence: str, n_words=70, deterministic=False):
    return next_words_best(sentence, lm, n_words, deterministic=deterministic)

In [16]:
for sentence in sentences:
    print("")
    print(prediction(sentence, deterministic=False))


Warszawa to największe miasto województwa podlaskiego .. dobrze Szymanko , Bytom , Fulgi .. najmniejsza aglomeracja miasta Włodawa .. bynajmniej

Te zabawki należą do naszej czerwonej szaletki staram się o doświadczonej dziewczynki czy wroc sekcyjnej .. bo widzę żyłki w Monice hatchback dużo ... ciasnym punkcie ciężkości okropnie zaburza głośność i ilość wibracji , jak w Team Fisichello ,

Policjant przygląda sięom tych posesji , przedstawiając własne opinie .. ale pozwoleniu na dali kościelne biskupstwo wspomnienia .. wychowane po tych wypadkach aresztowani .. dyskryminowałem kolesia ... wymordowanym naśladowcłoniełącznikiem .. 

Na środku skrzyżowania widać Kalwariębiciecznośćacjenci trzeźwą , przewlekle chorych poprzez pompowanie próchnicy ..modyfikowaneAsad zawiesile tresur Pojemność Źródeł awaryjne lotnisko na Dłużnicy 254 Wierzlska najświeższe oferty przetestowane w sezonie letnim 2005 popularne Okulary kolorowe Cały koncert do Twilight

Właściciel samochodu widział złodzieja z 

In [17]:
for sentence in sentences:
    print("")
    print(prediction(sentence, deterministic=True))


Warszawa to największe miasto w Polsce , które jest największym miastem w Polsce .. 

Te zabawki należą do najbardziej popularnych i najbardziej popularnych w Polsce .. 

Policjant przygląda się , jaka jest przyczyna tego , że w czasie pobytu w więzieniu , w którym przebywał , nie ma możliwości , by wylegitymować się z niego .. 

Na środku skrzyżowania widać ślady po 

Właściciel samochodu widział złodzieja z 

Prezydent z premierem rozmawiali wczoraj o tym , że w Polsce jest tak , że w Polsce jest ok. 50 tys. osób , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób ..

Witaj drogi , nie 

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie spodziewał , że to nie jest tak , że nie ma co się bać .. 

Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie spodziewała że to będzie jakiś żart .. 

Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę 